#### 1 - Inspect the scripts provided in the course page. Identify the layers of the neural network, as well as the relevant instructions for evaluating the MX-compatible formats. It is also recommended to read the specifications regarding the MX formats and data types, presented in this document (OCP Microscaling Formats (MX) Specification).

<small>

```
self.conv1 = mx.Conv2d(1, 32, 3, 1, mx_specs=mx_specs)
self.conv2 = mx.Conv2d(32, 64, 3, 1, mx_specs=mx_specs)
self.dropout1 = nn.Dropout(0.25)	
self.dropout2 = nn.Dropout(0.5)	
self.fc1 = mx.Linear(9216, 128, mx_specs=mx_specs)
self.fc2 = mx.Linear(128, 10, mx_specs=mx_specs)

(...)

x = self.conv1(x)
x = mx.relu(x, mx_specs=self.mx_specs)
x = self.conv2(x)
x = mx.relu(x, mx_specs=self.mx_specs)
x = F.max_pool2d(x, 2)              
x = self.dropout1(x)
x = torch.flatten(x, 1)             
x = self.fc1(x)
x = mx.relu(x, mx_specs=self.mx_specs)
x = self.dropout2(x)
x = self.fc2(x)
output = mx.simd_log(mx.softmax(x, dim=1, mx_specs=self.mx_specs), mx_specs=self.mx_specs)  
```
</small>

| Layer Order | Layer Type | Implementation | Configuration | MX-Compatible? |
| :--- | :--- | :--- | :--- | :---: |
| 1 | Convolution | `mx.Conv2d` | Input: 1, Output: 32, Kernel: 3x3 | Yes |
| 2 | Activation | `mx.relu` | Elementwise ReLU | Yes |
| 3 | Convolution | `mx.Conv2d` | Input: 32, Output: 64, Kernel: 3x3 | Yes |
| 4 | Activation | `mx.relu` | Elementwise ReLU | Yes |
| 5 | Pooling | `F.max_pool2d` | Max Pooling (2x2) | No |
| 6 | Dropout | `nn.Dropout` | Probability: 0.25 | No |
| 7 | Flatten | `torch.flatten` | Reshapes feature maps to vector | No |
| 8 | Fully Connected | `mx.Linear` | Input: 9216, Output: 128 | Yes |
| 9 | Activation | `mx.relu` | Elementwise ReLU | Yes |
| 10 | Dropout | `nn.Dropout` | Probability: 0.5 | No |
| 11 | Fully Connected | `mx.Linear` | Input: 128, Output: 10 | Yes |
| 12 | Output | `mx.softmax` + `mx.simd_log` | LogSoftmax | Yes |

In order to emulate different accelerators we need to change the mx_specs dictionary in the `main()` function of `pytorch_code_mx.py`, that will be called as the `mx_specs=` argument on the mx functions . 
The main configurations are:
*   **Format of matrix multiplication:**
    *   `mx_specs['w_elem_format']`: Weight precision
    *   `mx_specs['a_elem_format']`: Input activation precision.
*   **Elementwise Formats:** The precision for operations like ReLU and Softmax is controlled by:
    *   `mx_specs['bfloat']`: If non-zero, uses Brain Float format.
    *   `mx_specs['fp']`: If `bfloat` is `0`, this defines the total bit-width for a custom floating-point format (exponent is fixed at 5 bits). Otherwise it is ignored.
* **MX format:**
    *   `mx_specs['block_size']`: Set to 32 to use MX formats, every 32 elemtsn share one scale factor.
* **Shared scale:**
    *   `mx_specs['scale_bits']`: Set to 8 Bits for the shared scale factor.
* **CUDA acceleration:**
    *   `mx_specs['custom_cuda']`: True if we want CUDA acceleration.
